Installing the necessary libraries

In [85]:
!pip install faiss-cpu
!pip install sentence-transformers
!pip install groq
!pip install rich

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Importing the requirements

In [86]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from groq import Groq
import pickle

Defining the dataset to be stored to database

In [87]:
DATASET = {
    "Name": "TechShield",
    "Misson": "To empower Small and Medium Enterprises (SMEs) with robust cybersecurity solutions.",
    "Focus": "Addresses common SME challenges like limited resources, complex compliance, and inefficient incident response.",
    "Key Features": """
        AI-Powered Threat Detection & Prevention:
            - Real-time monitoring for anomalies and potential threats.
            - Automated patch management for vulnerabilities.
        Compliance as a Service (CaaS):
            - Simplified tracking of major regulations (e.g., GDPR, HIPAA, PCI-DSS).
            - Automated audit readiness reports.
        Incident Response & Management (IR&M):
            - Pre-configured response plans for common incidents.
            - 24/7 access to cybersecurity experts.
        User Awareness & Training:
            - Phishing simulations and training exercises.
            - Cybersecurity knowledge base and workshops.
    """,
    "Business Model":
        {
            "Revenue Streams": """
                - Subscription model with tiered pricing (Basic, Premium, Enterprise).
                - Enhanced support packages.
                - Custom compliance solutions.
                - Incident response retainer fees.
            """,
            "Marketing Strategies": """
                - Content marketing (white papers, webinars, case studies).
                - Partnerships with Managed Service Providers (MSPs) and industry associations.
                Targeted online advertising.
            """
        },
    "Development Roadmap": """
    - Phase 1 (Months 1-3): Team building, market research, platform design.
    - Phase 2 (Months 4-9): Development of core features (threat detection, CaaS, IR&M, training).
    - Phase 3 (Months 10-12): Testing, iteration, and launch preparation.
    - Launch & Growth (Quarter 2 onwards): Go-to-market, marketing campaigns, continuous improvement.
    """,   
}

Prepairing and preprocessing the dataset

In [88]:
def flatten_dict(d, parent_key='', sep=' '):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

flattened_dataset = flatten_dict(DATASET)

# Initialize the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2', cache_folder="./cache")

# Create embeddings for each item in the dataset
texts = list(flattened_dataset.values())
embeddings = model.encode(texts).astype('float32')

In [89]:
# Step 3: Store Data in Faiss
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Store the text and their corresponding embeddings
text_to_embedding = {text: embedding for text, embedding in zip(texts, embeddings)}


In [90]:
# Step 4: Save the embeddings and index
def save_embeddings_and_index(embeddings, index, text_to_embedding, embeddings_file='embeddings.pkl', index_file='faiss_index.bin'):
    # Save embeddings
    with open(embeddings_file, 'wb') as f:
        pickle.dump((texts, embeddings, text_to_embedding), f)
    
    # Save index
    faiss.write_index(index, index_file)

save_embeddings_and_index(embeddings, index, text_to_embedding)

In [91]:
# Step 5: Load the embeddings and index
def load_embeddings_and_index(embeddings_file='embeddings.pkl', index_file='faiss_index.bin'):
    # Load embeddings
    with open(embeddings_file, 'rb') as f:
        texts, embeddings, text_to_embedding = pickle.load(f)
    
    # Load index
    index = faiss.read_index(index_file)
    
    return texts, embeddings, text_to_embedding, index

texts, embeddings, text_to_embedding, index = load_embeddings_and_index()

In [92]:
# Step 6: Retrieve Data
def retrieve_data(query, index, text_to_embedding, top_k=1):
    # Create embedding for the query
    query_embedding = model.encode([query]).astype('float32')
    
    # Retrieve the most similar items
    distances, indices = index.search(query_embedding, top_k)
    
    # Get the corresponding texts
    retrieved_texts = [list(text_to_embedding.keys())[i] for i in indices[0]]
    
    return retrieved_texts


In [93]:
# Step 7: Process Retrieved Data with GROQ
GROQ_API_KEY = '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'

# Initialize the GROQ client
client = Groq(api_key=GROQ_API_KEY)

def generate_response(query, retrieved_texts):
    # Combine the query and retrieved texts
    system_prompt = f"here is the context provided to you for the query.\
        the context provided to you Is your own knowledge. you are working\
        as a reseptionist at a company and are providing answer to a person who is aking you\
        so keep your language in that manner. context:{retrieved_texts}"
    context = f"Query: {query}"
    
    # Generate response using GROQ (replace with actual GROQ API call)
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": context
            }
        ],
        model="llama3-8b-8192"
    )
    
    response = chat_completion.choices[0].message.content
    return response

In [94]:
query = "how does your business plan to earn?"
retrieved_texts = retrieve_data(query, index, text_to_embedding)
response = generate_response(query, retrieved_texts)

In [95]:
from rich.console import Console
from rich.markdown import Markdown

def md(text):
    console = Console()
    return console.print(Markdown(text))

In [96]:
print("Query:", query)
print("Retrieved Texts:", retrieved_texts)

Query: how does your business plan to earn?
Retrieved Texts: ['\n                - Content marketing (white papers, webinars, case studies).\n                - Partnerships with Managed Service Providers (MSPs) and industry associations.\n                Targeted online advertising.\n            ']


In [97]:
md(response)

At our company, we have a robust revenue strategy in place to ensure a steady influx of income. Firstly, we focus  
on content marketing efforts, specifically creating high-quality white papers, webinars, and case studies that     
provide immense value to our target audience. These valuable resources attract potential customers, and through    
targeted advertising, we direct them to our website and subsequently generate leads.                               

Additionally, we've established partnerships with Managed Service Providers (MSPs) and industry associations,      
creating a win-win situation for both parties. We offer expertise, and they bring in new clients. This synergy     
helps us expand our reach and improve our offerings.                                                               

Lastly, we utilize targeted online advertising to reach our ideal customer base, focusing on platforms that yield  
the highest conversions. By having a multi-channel approach, we're confident that our revenue streams will be      
diverse and consistent, allowing us to scale our operations and continue investing in growth initiatives.